<h1><center> Python Analytics Project </center></h1>

<center> <b><i>Team</i></b>: Karishma Velagaleti | Jiang Shang | Aditya Tripathi | Satyarth Poppasani </center>
<center> <b><i>Data Set </i></b>: HongKong Horse Racing Data-set</center>
<center> <b><i>Problem Statement</i></b>: Analytics for HongKong horse-race betting </center>

#### <center><b><i>Notebook 2</i>: TESTING MORE PREDICTIVE MODELS ON OUR DATA SET TO PREDICT FINISHING POSITIONS OF HORSES</b></center>

In [1]:
import pandas as pd
from pandas import Series
import numpy as np
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
import scipy.stats as stats
import sklearn
from sklearn import linear_model
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import BernoulliNB
import matplotlib.pyplot as plt
%matplotlib inline

/Users/karishmavelagaleti/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
results = pd.read_csv("race-result-horse.csv")
races = pd.read_csv("races_new.csv", encoding='latin-1')

In [3]:
results["horse_number"] = results["horse_number"].astype(str)
races["Month"] = races["Month"].astype(str)
races["Year"] = races["Year"].astype(str)

In [4]:
horseRace = pd.merge(results, races, on = "race_id", how = "inner")
horseRace["race_distance"] = horseRace["race_distance"].astype(str)

In [5]:
horseRace.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19996 entries, 0 to 19995
Data columns (total 31 columns):
finishing_position       19994 non-null object
horse_number             19996 non-null object
horse_name               19996 non-null object
horse_id                 19996 non-null object
jockey                   19996 non-null object
trainer                  19996 non-null object
actual_weight            19994 non-null float64
declared_horse_weight    19922 non-null float64
draw                     19595 non-null float64
length_behind_winner     19996 non-null object
running_position_1       19580 non-null float64
running_position_2       19575 non-null float64
running_position_3       19563 non-null float64
running_position_4       11117 non-null float64
finish_time              19996 non-null object
win_odds                 19595 non-null float64
running_position_5       2540 non-null float64
running_position_6       352 non-null float64
race_id                  19996 non-nul

In [6]:
df = pd.get_dummies(horseRace.ix[:,["jockey", "trainer","horse_id", "race_distance", "track", "race_class", "Month", "Year"]])
df1 = horseRace.ix[:, ["actual_weight"]]
Y = horseRace.ix[:,"finishing_position"]
df = df.reset_index()
df1 = df1.reset_index()
Y = Y.reset_index()
final = pd.concat([df,df1,Y],axis=1)
del final["index"]

In [7]:
final = final.dropna()
k = len(final.columns) - 1

In [9]:
X = final.ix[:,0:k]
Y = final.ix[:,["finishing_position"]]

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [11]:
X_train.head()

,jockey_---,jockey_A Atzeni,jockey_A Suborics,jockey_A de Vries,jockey_B Fayd'herbe,jockey_B Prebble,jockey_B Shinn,jockey_C Brown,jockey_C F Wong,jockey_C Hayes,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_9,Year_2014,Year_2015,Year_2016,actual_weight
13381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,130.0
19622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,119.0
482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,122.0
11300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,121.0
16918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,112.0


##### Decision Tree Classifier

In [12]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,Y_train)
print("Accuracy rate using Trees on Train: ", clf.score(X_train,Y_train))
pred = clf.predict(X_test)
pred = pd.DataFrame(pred, columns = ["prediction"])
Y_test = Y_test.reset_index()
del Y_test["index"]
new = pd.concat([pred, Y_test], axis = 1)
print("Accuracy rate using Trees on Test: ", clf.score(X_test, Y_test))

Accuracy rate using Trees on Train:  0.999928545909
Accuracy rate using Trees on Test:  0.082347057843


##### Boosting

In [13]:
clf = AdaBoostClassifier()
clf = clf.fit(X_train,Y_train)
print("Accuracy rate using Boosting on TRAINING: ", clf.score(X_train,Y_train))
print("Accuracy rate using Boosting on TEST: ", clf.score(X_test,Y_test))

/Users/karishmavelagaleti/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy rate using Boosting on TRAINING:  0.0924615934262
Accuracy rate using Boosting on TEST:  0.08468078013


##### Bagging 

In [14]:
base_clf = tree.DecisionTreeClassifier(max_depth=1)
clf = BaggingClassifier(n_estimators=50, base_estimator=base_clf, oob_score=True)
clf = clf.fit(X_train,Y_train)    
print("Accuracy rate using Bagging on TRAINING: ", clf.score(X_train, Y_train))
print("Accuracy rate using Bagging on TEST: ", clf.score(X_test, Y_test))

/Users/karishmavelagaleti/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy rate using Bagging on TRAINING:  0.0918185066095
Accuracy rate using Bagging on TEST:  0.0848474745791


##### Neural Network

In [15]:
X_train = X_train.ix[:,0:(k-1)] #neural net takes 0 and 1s values only - so we will stick with only dummy variables for this 
X_test = X_test.ix[:,0:(k-1)]
clf = BernoulliNB()
clf.fit(X_train, Y_train)
print("Accuracy using Neural Net for TRAINING: ", clf.score(X_train, Y_train))
print("Accuracy using Neural Net for TEST: ", clf.score(X_test, Y_test))

/Users/karishmavelagaleti/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy using Neural Net for TRAINING:  0.27173990711
Accuracy using Neural Net for TEST:  0.111185197533


In [16]:
X_train.head()

,jockey_---,jockey_A Atzeni,jockey_A Suborics,jockey_A de Vries,jockey_B Fayd'herbe,jockey_B Prebble,jockey_B Shinn,jockey_C Brown,jockey_C F Wong,jockey_C Hayes,...,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_9,Year_2014,Year_2015,Year_2016
13381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
11300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
16918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
